In [ ]:
import polars as pl
import bpy
#from bl_ext.blender_org.csv_importer import PolarsMesh


In [19]:
# based on https://github.com/BradyAJohnston/MolecularNodes/blob/80f3dac339750638bb5f5edc3b1e66292fccf642/molecularnodes/bpyd/attribute.py
# and this prompt: https://chatgpt.com/share/672de0c6-b18c-8013-84d1-ecf23043c790
import bpy
import numpy as np
import polars as pl
from dataclasses import dataclass
from typing import Type, Any
from enum import Enum

class DomainType(Enum):
    POINT = "POINT"
    EDGE = "EDGE"
    FACE = "FACE"
    CORNER = "CORNER"
    CURVE = "CURVE"
    INSTANCE = "INSTANCE"
    LAYER = "LAYER"

class Domains:
    POINT = DomainType.POINT
    EDGE = DomainType.EDGE
    FACE = DomainType.FACE
    CORNER = DomainType.CORNER
    CURVE = DomainType.CURVE
    INSTANCE = DomainType.INSTANCE
    LAYER = DomainType.LAYER

@dataclass
class AttributeType:
    type_name: str
    value_name: str
    dtype: Type
    dimensions: tuple

    def __str__(self) -> str:
        return self.type_name

class AttributeTypes(Enum):
    FLOAT = AttributeType("FLOAT", "value", float, (1,))
    FLOAT_VECTOR = AttributeType("FLOAT_VECTOR", "vector", float, (3,))
    FLOAT2 = AttributeType("FLOAT2", "vector", float, (2,))
    FLOAT_COLOR = AttributeType("FLOAT_COLOR", "color", float, (4,))
    BYTE_COLOR = AttributeType("BYTE_COLOR", "color", int, (4,))
    QUATERNION = AttributeType("QUATERNION", "value", float, (4,))
    INT = AttributeType("INT", "value", int, (1,))
    INT8 = AttributeType("INT8", "value", int, (1,))
    INT32_2D = AttributeType("INT32_2D", "value", int, (2,))
    FLOAT4X4 = AttributeType("FLOAT4X4", "matrix", float, (4, 4))
    BOOLEAN = AttributeType("BOOLEAN", "value", bool, (1,))

def guess_attribute_type(series: pl.Series) -> AttributeType:
    print(f"[DEBUG guess_attribute_type] Start: {series.name}, dtype={series.dtype}")
    dtype = series.dtype
    if dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64]:
        print("[DEBUG guess_attribute_type] Detected INT type")
        return AttributeTypes.INT.value
    elif dtype in [pl.Float32, pl.Float64]:
        print("[DEBUG guess_attribute_type] Detected FLOAT type")
        return AttributeTypes.FLOAT.value
    elif dtype == pl.Boolean:
        print("[DEBUG guess_attribute_type] Detected BOOLEAN type")
        return AttributeTypes.BOOLEAN.value
    elif isinstance(dtype, pl.datatypes.List):
        print("[DEBUG guess_attribute_type] dtype is a LIST, checking first valid entry")
        first_valid = None
        for val in series:
            if val is not None:
                first_valid = val
                break
        print(f"[DEBUG guess_attribute_type] First valid in list: {first_valid}")
        if first_valid is not None:
            # If the first_valid is a pl.Series, handle similarly to object columns
            if isinstance(first_valid, pl.Series):
                length = len(first_valid)
                print(f"[DEBUG guess_attribute_type] Detected Series in list with length {length}")
                if length == 2:
                    return AttributeTypes.FLOAT2.value
                elif length == 3:
                    return AttributeTypes.FLOAT_VECTOR.value
                elif length == 4:
                    return AttributeTypes.FLOAT_COLOR.value
                elif length == 16:
                    return AttributeTypes.FLOAT4X4.value
                else:
                    print("[DEBUG guess_attribute_type] Unsupported length for series type in list, falling back to FLOAT_VECTOR")
                    return AttributeTypes.FLOAT_VECTOR.value
            # Else assume a numeric list
            elif isinstance(first_valid, (list, tuple, np.ndarray)):
                length = len(first_valid)
                print(f"[DEBUG guess_attribute_type] Length of first_valid numeric list: {length}")
                if length == 2:
                    return AttributeTypes.FLOAT2.value
                elif length == 3:
                    return AttributeTypes.FLOAT_VECTOR.value
                elif length == 4:
                    return AttributeTypes.FLOAT_COLOR.value
                elif length == 16:
                    return AttributeTypes.FLOAT4X4.value
                else:
                    print("[DEBUG guess_attribute_type] Unsupported length for numeric list, falling back to FLOAT_VECTOR")
                    return AttributeTypes.FLOAT_VECTOR.value
    elif dtype == pl.Object:
        print("[DEBUG guess_attribute_type] dtype is OBJECT, checking for pl.Series in cells")
        first_valid = None
        for val in series:
            if val is not None:
                first_valid = val
                break
        print(f"[DEBUG guess_attribute_type] First valid in object column: {first_valid}")
        if first_valid is not None and isinstance(first_valid, pl.Series):
            length = len(first_valid)
            print(f"[DEBUG guess_attribute_type] Detected Series in cells with length {length}")
            if length == 2:
                return AttributeTypes.FLOAT2.value
            elif length == 3:
                return AttributeTypes.FLOAT_VECTOR.value
            elif length == 4:
                return AttributeTypes.FLOAT_COLOR.value
            elif length == 16:
                return AttributeTypes.FLOAT4X4.value
            else:
                print("[DEBUG guess_attribute_type] Unsupported length for series type, falling back to FLOAT_VECTOR")
                return AttributeTypes.FLOAT_VECTOR.value
    # Default fallback
    print("[DEBUG guess_attribute_type] Fallback to FLOAT")
    return AttributeTypes.FLOAT.value

class PolarsMesh:
    def __init__(self, dataframe: pl.DataFrame, mesh_name: str = "PointCloudMeshwithAttributes", object_name: str = "PointCloudAttributes"):
        print("[DEBUG PolarsMesh.__init__] Initializing PolarsMesh")
        self.mesh_name = mesh_name
        self.object_name = object_name
        self.included_columns = []
        self.process_dataframe(dataframe)
        self.length = len(self.dataframe)
        print(f"[DEBUG PolarsMesh.__init__] Length of dataframe: {self.length}")
        self.vertices = [(0, 0, 0)] * self.length
        self.mesh = bpy.data.meshes.new(self.mesh_name)
        self.point_obj = bpy.data.objects.new(self.object_name, self.mesh)
        print("[DEBUG PolarsMesh.__init__] Created mesh and object")
        # Uncomment if running in Blender
        # bpy.context.collection.objects.link(self.point_obj)
        print("[DEBUG PolarsMesh.__init__] Creating mesh from pydata")
        self.mesh.from_pydata(self.vertices, [], [])
        self.mesh.update()
        print("[DEBUG PolarsMesh.__init__] Mesh updated, adding attributes")
        self.add_attributes()

    def process_dataframe(self, dataframe: pl.DataFrame):
        print("[DEBUG process_dataframe] Processing dataframe")
        included_columns = []
        excluded_columns = []
        for col, dtype in zip(dataframe.columns, dataframe.dtypes):
            print(f"[DEBUG process_dataframe] Checking column: {col}, dtype={dtype}")
            if dtype in [pl.Boolean, pl.Float32, pl.Float64, pl.Int8, pl.Int16, pl.Int32, pl.Int64]:
                included_columns.append(col)
            elif isinstance(dtype, pl.datatypes.List):
                print("[DEBUG process_dataframe] Column is list type, checking first valid")
                first_valid = None
                for val in dataframe[col]:
                    if val is not None:
                        first_valid = val
                        break
                print(f"[DEBUG process_dataframe] first_valid: {first_valid}")
                if first_valid is not None:
                    if isinstance(first_valid, pl.Series):
                        # Check that all are either None or Series of the same length
                        length = len(first_valid)
                        if all((isinstance(v, pl.Series) and len(v) == length) or v is None for v in dataframe[col]):
                            included_columns.append(col)
                        else:
                            excluded_columns.append(col)
                    elif isinstance(first_valid, (list, tuple, np.ndarray)):
                        # Numeric lists
                        included_columns.append(col)
                    else:
                        excluded_columns.append(col)
                else:
                    excluded_columns.append(col)
            elif dtype == pl.Object:
                print("[DEBUG process_dataframe] Column is object type, checking for pl.Series in cells")
                first_valid = None
                for val in dataframe[col]:
                    if val is not None:
                        first_valid = val
                        break
                print(f"[DEBUG process_dataframe] first_valid: {first_valid}")
                if first_valid is not None and isinstance(first_valid, pl.Series):
                    length = len(first_valid)
                    print(f"[DEBUG process_dataframe] First valid series length: {length}")
                    if all((isinstance(v, pl.Series) and len(v) == length) or v is None for v in dataframe[col]):
                        included_columns.append(col)
                    else:
                        excluded_columns.append(col)
                else:
                    excluded_columns.append(col)
            else:
                excluded_columns.append(col)

        if excluded_columns:
            print(f"Columns not included (unsupported data types): {excluded_columns}")
        self.dataframe = dataframe.select(included_columns)
        self.included_columns = included_columns
        print(f"[DEBUG process_dataframe] Columns added to the mesh: {included_columns}")

    def add_attributes(self):
        print("[DEBUG add_attributes] Adding attributes for included columns")
        for column in self.dataframe.columns:
            print(f"[DEBUG add_attributes] Processing column: {column}")
            series = self.dataframe[column]
            attr_type = guess_attribute_type(series)
            print(f"[DEBUG add_attributes] Attribute type guessed: {attr_type.type_name}")
            data = self.series_to_numpy(series, attr_type)
            print(f"[DEBUG add_attributes] Data shape after conversion: {data.shape}")
            self.store_attribute(self.point_obj, data, column, attr_type)

    def update(self, dataframe: pl.DataFrame):
        print("[DEBUG update] Updating with new dataframe")
        previous_columns = set(self.included_columns)
        self.process_dataframe(dataframe)
        new_length = len(self.dataframe)
        if new_length != self.length:
            print(f"[DEBUG update] Dataframe length changed from {self.length} to {new_length}, updating mesh vertices.")
            self.length = new_length
            self.vertices = [(0, 0, 0)] * self.length
            self.mesh.from_pydata(self.vertices, [], [])
            self.mesh.update()

        # Remove previous attributes
        for attr_name in previous_columns:
            print(f"[DEBUG update] Removing old attribute: {attr_name}")
            if attr_name in self.mesh.attributes:
                self.mesh.attributes.remove(self.mesh.attributes[attr_name])

        # Add new attributes
        print("[DEBUG update] Adding new attributes")
        self.add_attributes()

    def clear(self):
        print("[DEBUG clear] Clearing all attributes")
        self.dataframe = pl.DataFrame()
        for attr_name in self.included_columns:
            print(f"[DEBUG clear] Removing attribute: {attr_name}")
            if attr_name in self.mesh.attributes:
                self.mesh.attributes.remove(self.mesh.attributes[attr_name])
        self.included_columns = []

    def store_attribute(self, obj: bpy.types.Object, data: np.ndarray, name: str, attr_type: AttributeType, domain: DomainType = DomainType.POINT):
        print(f"[DEBUG store_attribute] Storing attribute '{name}' with type '{attr_type.type_name}' and domain '{domain.value}'")
        attributes = obj.data.attributes
        if name in attributes:
            attribute = attributes[name]
            print(f"[DEBUG store_attribute] Attribute '{name}' already exists, reusing it.")
        else:
            attribute = attributes.new(name, attr_type.type_name, domain.value)
            print(f"[DEBUG store_attribute] Created new attribute '{name}'")

        expected_length = len(attribute.data) * int(np.prod(attr_type.dimensions))
        print(f"[DEBUG store_attribute] expected_length={expected_length}, data.size={data.size}")
        if data.size != expected_length:
            raise ValueError(f"Data length {data.size} does not match expected size {expected_length} for attribute '{name}'.")
        attribute.data.foreach_set(attr_type.value_name, data.flatten())
        print(f"[DEBUG store_attribute] Finished setting data for attribute '{name}'")

    def series_to_numpy(self, series: pl.Series, attr_type: AttributeType) -> np.ndarray:
        print(f"[DEBUG series_to_numpy] Converting series '{series.name}' to numpy for attribute type '{attr_type.type_name}'")
        # Handle simple scalar attributes
        if attr_type.dimensions == (1,):
            arr = series.to_numpy().astype(attr_type.dtype)
            print(f"[DEBUG series_to_numpy] Scalar attribute, arr.shape={arr.shape}")
            return arr

        # Handle lists or series of arrays
        if isinstance(series.dtype, pl.datatypes.List):
            # The column might be lists of either numeric arrays or pl.Series
            arr_list = []
            for val in series:
                if val is None:
                    arr_list.append(np.zeros(attr_type.dimensions, dtype=attr_type.dtype))
                elif isinstance(val, pl.Series):
                    arr_list.append(val.to_numpy())
                else:
                    arr_list.append(np.array(val, dtype=attr_type.dtype))
            arr = np.array(arr_list, dtype=attr_type.dtype)
            print(f"[DEBUG series_to_numpy] LIST column converted, arr.shape={arr.shape}")
        elif series.dtype == pl.Object:
            print("[DEBUG series_to_numpy] Column is OBJECT, converting from pl.Series in each cell")
            arr_list = []
            for val in series:
                if val is not None:
                    val_arr = val.to_numpy()
                    arr_list.append(val_arr)
                else:
                    # If there are missing values
                    arr_list.append(np.zeros(attr_type.dimensions, dtype=attr_type.dtype))
            arr = np.array(arr_list, dtype=attr_type.dtype)
            print(f"[DEBUG series_to_numpy] OBJECT column converted, arr.shape={arr.shape}")
        else:
            # dtype is likely a numeric list
            print("[DEBUG series_to_numpy] Column is numeric LIST or vector-like, using to_list()")
            arr = np.array(series.to_list(), dtype=attr_type.dtype)
            print(f"[DEBUG series_to_numpy] arr.shape before reshape={arr.shape}")

        if arr.ndim == 1:
            print("[DEBUG series_to_numpy] Expanding dimensions since arr.ndim=1")
            arr = np.expand_dims(arr, axis=1)

        final_shape = (-1, int(np.prod(attr_type.dimensions)))
        arr = arr.reshape(final_shape)
        print(f"[DEBUG series_to_numpy] Final arr.shape={arr.shape}")
        return arr

In [26]:

# Read the JSON file
df = pl.read_json("/Users/jan-hendrik/projects/blender_csv_import/generate_data/dino_star_vectors_3d_vector.json")
# Identify columns with list[list] values
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
# Explode all identified columns
df = df.explode(columns_to_explode)

blender_mesh = PolarsMesh(dataframe=df, object_name=f"JSON OBJ")

# Link the new mesh to the Blender scene
bpy.context.collection.objects.link(blender_mesh.point_obj)

[DEBUG PolarsMesh.__init__] Initializing PolarsMesh
[DEBUG process_dataframe] Processing dataframe
[DEBUG process_dataframe] Checking column: Dino, dtype=List(Float64)
[DEBUG process_dataframe] Column is list type, checking first valid
[DEBUG process_dataframe] first_valid: shape: (3,)
Series: '' [f64]
[
	55.3846
	97.1795
	0.0
]
[DEBUG process_dataframe] Checking column: Star, dtype=List(Float64)
[DEBUG process_dataframe] Column is list type, checking first valid
[DEBUG process_dataframe] first_valid: shape: (3,)
Series: '' [f64]
[
	58.2136
	91.8819
	0.0
]
[DEBUG process_dataframe] Columns added to the mesh: ['Dino', 'Star']
[DEBUG PolarsMesh.__init__] Length of dataframe: 142
[DEBUG PolarsMesh.__init__] Created mesh and object
[DEBUG PolarsMesh.__init__] Creating mesh from pydata
[DEBUG PolarsMesh.__init__] Mesh updated, adding attributes
[DEBUG add_attributes] Adding attributes for included columns
[DEBUG add_attributes] Processing column: Dino
[DEBUG guess_attribute_type] Start: Din

In [23]:
df

Dino,Star
list[list[f64]],list[list[f64]]
"[[55.3846, 97.1795], [51.5385, 96.0256], … [44.1026, 92.6923]]","[[58.2136, 91.8819], [58.1961, 92.215], … [58.2432, 92.1043]]"


In [21]:
import polars as pl
import bpy

# Read the JSON file
df = pl.read_json("/Users/jan-hendrik/projects/blender_csv_import/generate_data/dino_star_vectors.json")

# First, explode the nested lists
df = df.explode(["Dino", "Star"])

# Convert the lists to proper Series format
df = df.with_columns([
    pl.col("Dino").map_elements(lambda x: pl.Series(x)).alias("Dino"),
    pl.col("Star").map_elements(lambda x: pl.Series(x)).alias("Star")
])

# Create and link the mesh
blender_mesh = PolarsMesh(dataframe=df, object_name="JSON OBJ")
bpy.context.collection.objects.link(blender_mesh.point_obj)

[DEBUG PolarsMesh.__init__] Initializing PolarsMesh
[DEBUG process_dataframe] Processing dataframe
[DEBUG process_dataframe] Checking column: Dino, dtype=List(Float64)
[DEBUG process_dataframe] Column is list type, checking first valid
[DEBUG process_dataframe] first_valid: shape: (2,)
Series: '' [f64]
[
	55.3846
	97.1795
]
[DEBUG process_dataframe] Checking column: Star, dtype=List(Float64)
[DEBUG process_dataframe] Column is list type, checking first valid
[DEBUG process_dataframe] first_valid: shape: (2,)
Series: '' [f64]
[
	58.2136
	91.8819
]
[DEBUG process_dataframe] Columns added to the mesh: ['Dino', 'Star']
[DEBUG PolarsMesh.__init__] Length of dataframe: 142
[DEBUG PolarsMesh.__init__] Created mesh and object
[DEBUG PolarsMesh.__init__] Creating mesh from pydata
[DEBUG PolarsMesh.__init__] Mesh updated, adding attributes
[DEBUG add_attributes] Adding attributes for included columns
[DEBUG add_attributes] Processing column: Dino
[DEBUG guess_attribute_type] Start: Dino, dtype=L

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [17]:
type(df["Dino"] [0])

polars.series.series.Series

In [18]:
df["Dino"] 

Dino
list[f64]
"[55.3846, 97.1795]"
"[51.5385, 96.0256]"
"[46.1538, 94.4872]"
"[42.8205, 91.4103]"
"[40.7692, 88.3333]"
…
"[39.4872, 25.3846]"
"[91.2821, 41.5385]"
"[50.0, 95.7692]"


In [6]:
import polars as pl

# Ensure each cell remains as a list
df = df.with_columns(
    pl.col("Dino").map_elements(lambda x: list(x)).alias("Dino")
)

print(df)

shape: (142, 2)
┌────────────────────┬────────────────────┐
│ Dino               ┆ Star               │
│ ---                ┆ ---                │
│ list[f64]          ┆ list[f64]          │
╞════════════════════╪════════════════════╡
│ [55.3846, 97.1795] ┆ [58.2136, 91.8819] │
│ [51.5385, 96.0256] ┆ [58.1961, 92.215]  │
│ [46.1538, 94.4872] ┆ [58.7182, 90.3105] │
│ [42.8205, 91.4103] ┆ [57.2784, 89.9076] │
│ [40.7692, 88.3333] ┆ [58.082, 92.0081]  │
│ …                  ┆ …                  │
│ [39.4872, 25.3846] ┆ [43.7226, 19.0773] │
│ [91.2821, 41.5385] ┆ [79.3261, 52.9004] │
│ [50.0, 95.7692]    ┆ [56.664, 87.9401]  │
│ [47.9487, 95.0]    ┆ [57.8218, 90.6932] │
│ [44.1026, 92.6923] ┆ [58.2432, 92.1043] │
└────────────────────┴────────────────────┘


/var/folders/0n/w5m51rrn71db4cvkds08wdg80000gn/T/ipykernel_1174/2906863085.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


In [ ]:
type(df["Dino"] [0])